In [17]:
!pip install -q bitsandbytes datasets accelerate loralib openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.0 MB/s eta 0:00:00


#We want the dataset in the form of 2 columns, first being the diary entry and the second being the rap song.

In [32]:
import random

age_choices = tuple(range(12,65))
gender_choices = ('male', 'female')
mood_choices = ('great', 'good', 'bad', 'annoyed', 'exhausted', 'happy', 'sad', 'heartbreak', 'loving')
writing_choices = ('professional', 'formal', 'informal', 'talkative', 'expressive', 'introverted')

def get_choices():
  print("Inside get_choices")
  age = random.choice(age_choices)
  gender = random.choice(gender_choices)
  print("age,gender: ", age,gender)
  mood = random.choice(mood_choices)
  writing = random.choice(writing_choices)

  return (age,gender,mood, writing)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain import LLMChain
import os
from langchain.llms import OpenAI

os.environ['OPENAI_API_KEY'] = ""
llm = OpenAI(model_name='gpt-3.5-turbo-16k', temperature=1)

writing_manner_array = []
mood_array = []
gender_array = []
age_array = []
diary_entry_array = []
rap_song_array = []
def get_examples():
  propmt_template_diary_entry = PromptTemplate(
          input_variables=["Age", "Gender", "Mood", "Writing_manner"],
          template="Write a diary entry around 150 words on behalf of a {Age} year old {Gender} who is in a {Mood} mood. Assume this person writes in a {Writing_manner} way."
      )


  diary_entry_chain = LLMChain(llm=llm, prompt=propmt_template_diary_entry, output_key="diary_entry")

  propmt_template_rap = PromptTemplate(
      input_variables=["diary_entry"],
      template="Write a short rap song based on the diary entry given below. Make sure the rap song has a motivational ending. The diary entry is: {diary_entry}"
  )

  rap_song_chain = LLMChain(llm=llm, prompt=propmt_template_rap, output_key="rap_song")

  chain = SequentialChain(
      chains=[diary_entry_chain, rap_song_chain],
      input_variables=["Age", "Gender", "Mood", "Writing_manner"],
      output_variables=["diary_entry", "rap_song"]
  )
  print("Chain created successfully.")
  age, gender, mood, writing_manner = get_choices()
  print(age,gender,mood,writing_manner)
  resp = chain({"Age": age, "Gender": gender, "Mood": mood, "Writing_manner": writing_manner})
  print("resp is: ", resp)
  age_array.append(resp['Age'])
  mood_array.append(resp['Mood'])
  gender_array.append(resp['Gender'])
  diary_entry_array.append(resp['diary_entry'])
  rap_song_array.append(resp['rap_song'])
  writing_manner_array.append(resp['Writing_manner'])

for i in range(0,250):
  get_examples()

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Chain created successfully.
Inside get_choices
age,gender:  19 female
19 female exhausted introverted
resp is:  {'Age': 19, 'Gender': 'female', 'Mood': 'exhausted', 'Writing_manner': 'introverted', 'diary_entry': 'Dear Diary,\n\nI find solace in your pages today, as my weary soul seeks refuge from the relentless demands of the world. Fatigue has woven its heavy tendrils around my body, stifling my spirit and clouding my mind. The weariness seems never-ending, suffocating every ounce of energy that once resided within me.\n\nThe incessant noise and constant motion of life have suffused my being, leaving me utterly drained. The weight of expectations and responsibilities bears down upon my shoulders, making each step feel like a monumental effort. It feels as if I am eternally trapped in a labyrinth of tasks, forever searching for an escape that eludes me.\n\nIn this moment, all I desire is solitude; a chance to retreat into the sanctuary of my introverted nature. To find solace within m

resp is:  {'Age': 53, 'Gender': 'male', 'Mood': 'happy', 'Writing_manner': 'introverted', 'diary_entry': "Dear Diary,\n\nToday has been a rather delightful day. The sun's warm rays gently illuminated my room, coaxing me out of bed with a sense of contentment. As I sipped my morning coffee, a profound sense of gratitude washed over me, and I couldn't help but smile.\n\nThe simplicity of this day has brought me immense joy. The solitude I find myself in has become a cherished companion, allowing me to fully appreciate the little things in life that often go unnoticed. The soft hum of nature outside my window, the familiar scent of fresh air, these simple pleasures always manage to nudge my heart.\n\nI've had the opportunity to lose myself in the pages of a captivating book, escaping to distant lands and immersing myself in otherworldly adventures. As the hours slipped away, I felt an indescribable satisfaction, as if I had discovered a hidden treasure within the depths of my quiet sanctu

resp is:  {'Age': 24, 'Gender': 'female', 'Mood': 'annoyed', 'Writing_manner': 'informal', 'diary_entry': 'Dear Diary,\n\nUgh, so today has not been my day. I\'m in such an annoyed mood, I can\'t even begin to explain it. Everything seems to be going wrong, you know? First of all, I wake up to the sound of my alarm blaring in my ears, and it\'s like the universe is mocking me, saying, "Hey, guess what? Another day of stress and headaches for you!"\n\nAnd then, my coffee machine decides to go on strike. Seriously? I need my caffeine fix to even function properly. But no, it just decides to give up on me today. Fantastic. So, off I go to work, already irritated and running late. Traffic? Oh, don\'t even get me started. It\'s like all the slow drivers in the world gathered on the road just to tick me off. Not to mention, my boss decides today is the perfect day to breathe down my neck and micromanage every little thing I do.\n\nI need a vacation, Diary. Somewhere far, far away from all th

resp is:  {'Age': 23, 'Gender': 'male', 'Mood': 'good', 'Writing_manner': 'professional', 'diary_entry': "Dear Diary,\n\nToday has been an absolutely fantastic day that has filled me with an immense sense of joy and contentment. It's one of those rare days when everything seems to fall perfectly into place, and I can't help but embrace this incredible positive energy that surrounds me.\n\nI woke up this morning feeling refreshed and rejuvenated, ready to conquer the world. The sun was shining ever so brightly, casting its warm rays through the window, spreading a golden glow inside my room. The atmosphere seemed to be exuding nothing but optimism and endless possibilities.\n\nAs I made my way to work, my favorite song played on the radio, lifting my spirits even higher. The commute flew by, and stepping into the office, I was greeted by my colleagues with wide smiles and genuine warmth. The energy within our team was palpable, and together we achieved remarkable milestones, confidently

In [37]:
print(age_array, mood_array, gender_array, diary_entry_array, rap_song_array, writing_manner_array)

[42] ['great'] ['female'] ["Dear Diary,\n\nOh boy, let me tell you, today was a day for the books! I am bubbling over with joy, I can hardly contain it! It's one of those days when the sunshine seems brighter, the birds chirp louder, and even the butterflies in my stomach are dancing with elation. I'm 42 years young, and honey, I feel like I have the whole world at my fingertips!\n\nFrom the moment that alarm clock went off and whisked me out of dreamland, I knew something special was in the air. The universe was conspiring in my favor, I swear! Everything just clicked into place - from nailing that big presentation at work to treating myself to a delightful latte at my favorite café. The energy was contagious, and it seemed like everyone I crossed paths with had an extra bounce in their step.\n\nAs I reflect on this exceptional day, I can't help but grin from ear to ear. Life is meant to be embraced with gusto, and today was a shining example of that. Now, I'm off to celebrate this en

In [38]:
import pandas as pd

series_age_array = pd.Series(age_array)
series_gender_array = pd.Series(gender_array)
series_mood_array = pd.Series(mood_array)
series_writing_manner_array = pd.Series(writing_manner_array)
series_diary_entry_array = pd.Series(diary_entry_array)
series_rap_song_array = pd.Series(rap_song_array)

In [45]:
concatenated_series = pd.concat([series_age_array, series_gender_array, series_mood_array,series_writing_manner_array ,series_diary_entry_array ,series_rap_song_array],axis = 1,
                                keys=['Age', 'Gender', 'Mood', 'Writing Manner', 'Diary Entry', 'Rap Song'])


In [46]:
concatenated_series

,Age,Gender,Mood,Writing Manner,Diary Entry,Rap Song
0,42,female,great,talkative,"Dear Diary,\n\nOh boy, let me tell you, today ...",(Verse 1)\nI woke up this morning with a feeli...
